In [1]:
import numpy as np
import scipy.optimize as opt
from scipy.stats import entropy


In [2]:

# Generate synthetic data
np.random.seed(0)
n, k = 100, 3
X = np.random.randn(n, k)
beta_true = np.array([1.5, -2.0, 0.5])
epsilon = np.random.randn(n)
y = X @ beta_true + epsilon


In [3]:

# OLS Estimator
beta_ols = np.linalg.inv(X.T @ X) @ X.T @ y


In [6]:

# ME Estimator
# Define the discrete support space for errors
r = 5
e_support = np.linspace(-3, 3, r)

def me_objective(q, y, X, beta, e_support):
    # Calculate the errors
    epsilon_est = np.dot(q.reshape((n, r)), e_support)
    # Calculate the residuals
    residuals = y - X @ beta - epsilon_est
    # Calculate the entropy
    q = q.reshape((n, r))
    return -np.sum(entropy(q.T))

# Constraints
def me_constraints(q, y, X, beta, e_support):
    q = q.reshape((n, r))
    epsilon_est = np.dot(q, e_support)
    return y - X @ beta - epsilon_est

# Initial guess
q0 = np.full((n * r), 1/r)

# Bounds for q
bounds = [(0, 1) for _ in range(n * r)]

# Constraints for optimization
constraints = [{'type': 'eq', 'fun': me_constraints, 'args': (y, X, beta_ols, e_support)},
               {'type': 'eq', 'fun': lambda q: np.sum(q.reshape((n, r)), axis=1) - 1}]

# Optimization
result = opt.minimize(me_objective, q0, args=(y, X, beta_ols, e_support),
                      method='SLSQP', bounds=bounds, constraints=constraints)


In [8]:

# Calculate ME estimator errors
q_opt = result.x.reshape((n, r))
epsilon_me = np.dot(q_opt, e_support)

# Adjust y to remove the estimated errors for beta estimation
y_adjusted = y - epsilon_me
beta_me = np.linalg.inv(X.T @ X) @ X.T @ y_adjusted

# Compare the results
print("True Beta:", beta_true)
print("OLS Beta:", beta_ols)
print("ME Beta:", beta_me)
print("ME Beta:", q_opt)

True Beta: [ 1.5 -2.   0.5]
OLS Beta: [ 1.4371939  -2.07089475  0.56618568]
ME Beta: [ 1.4371939  -2.07089475  0.56618568]
ME Beta: [[0.40262604 0.2580451  0.16539847 0.10599225 0.06793813]
 [0.01906018 0.04481781 0.10540518 0.24785928 0.58285755]
 [0.208406   0.2041152  0.19991334 0.19579806 0.1917674 ]
 [0.31172148 0.24189394 0.18769759 0.14565724 0.11302975]
 [0.1178677  0.14934408 0.18922102 0.23976849 0.30379871]
 [0.24503641 0.22011019 0.19771412 0.17760364 0.15953564]
 [0.3974237  0.25751752 0.16687659 0.10811643 0.07006576]
 [0.39525106 0.25728394 0.16748729 0.10901042 0.07096729]
 [0.11407578 0.14646223 0.18803827 0.24143887 0.30998485]
 [0.2218972  0.21036435 0.19943056 0.1890665  0.17924138]
 [0.20515297 0.20254359 0.19996733 0.19742368 0.19491243]
 [0.09554646 0.1315037  0.18098932 0.24910811 0.34285242]
 [0.34653461 0.24984336 0.1801254  0.12986611 0.09363052]
 [0.30995526 0.24143076 0.18804405 0.14647608 0.11409385]
 [0.24499955 0.22009564 0.19771751 0.17762157 0.15956573

In [ ]:

# Initial guess
q0 = np.full((n, r), 1/r)

# Bounds for q
bounds = [(0, 1) for _ in range(n * r)]

# Constraints for optimization
constraints = [{'type': 'eq', 'fun': me_constraints, 'args': (y, X, beta_ols, e_support)},
               {'type': 'eq', 'fun': lambda q: np.sum(q.reshape((n, r)), axis=1) - 1}]

# Optimization
result = opt.minimize(me_objective, q0, args=(y, X, beta_ols, e_support),
                      method='SLSQP', bounds=bounds, constraints=constraints)

# Calculate ME estimator errors
q_opt = result.x.reshape((n, r))
epsilon_me = np.dot(q_opt, e_support)

# Adjust y to remove the estimated errors for beta estimation
y_adjusted = y - epsilon_me
beta_me = np.linalg.inv(X.T @ X) @ X.T @ y_adjusted

# Compare the results
print("True Beta:", beta_true)
print("OLS Beta:", beta_ols)
print("ME Beta:", beta_me)
